<a href="https://colab.research.google.com/github/kingjiwoo/nlpbible/blob/main/LSTM_%EB%84%A4%ED%8A%B8%EC%9B%8C%ED%81%AC%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file

import numpy as np
import random
import sys
import io

In [2]:
# 텍스트 파일 불러오기
fpath = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(fpath, encoding='utf-8') as f:
    text = f.read().lower()

600901/600901 [==============================] - 0s 0us/step


In [3]:
sorted(list(set(text)))

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'ä',
 'æ',
 'é',
 'ë']

In [4]:
#어휘 사전 생성
chars = sorted(list(set(text)))
char2index = dict((c,i) for i,c in enumerate(chars))
index2char = dict((i,c) for i,c in enumerate(chars))

In [5]:
#음절 단위 학습 데이터 생성
maxlen, step = 40, 3
sentences, next_chars = [], []
for i in range(0,len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print('The number of sentences:', len(sentences))
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i,sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t,char2index[char]] = 1
    y[i, char2index[next_chars[i]]] = 1

The number of sentences: 200285


<ipython-input-5-fb8a0a1b848a>:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-5-fb8a0a1b848a>:9: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [6]:
#딥러닝 모델 선언
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
# 입력된 확률값에 따른 다음 음절 샘플링
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 1회 (1 epoch) 학습
def on_epoch_end(epoch, _):
    print('\nEpoch: %d' % epoch)
    start_index = random.randint(0, len (text) - maxlen - 1 )
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('\nDiversity:', diversity)
        generated = ''
        sentence = text[start_index : start_index + maxlen]
        generated += sentence
        print('Seed: %s' % sentence)
        sys.stdout.write(generated)
        for i in range (400 ):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char2index[char]] = 1.
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = index2char[next_index]
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
1561/1565 [============================>.] - ETA: 0s - loss: 2.0163
Epoch: 0

Diversity: 0.2
Seed: termined by different moralities; our ac
termined by different moralities; our acture and the man ever to the sear of the searly to the period, and have in the sears to the sear of the sear of the is a such to the mander and the will as the sears of the sear of the sear of the sears of the way that is to the sears and and to the such to the was as the intertain of the sear of the and be the sear of the sear of the sear of the have is all that the have and the sear of the sear 
Diversity: 0.5
Seed: termined by different moralities; our ac
termined by different moralities; our actull of the stands may be least fart to all to that it is it is to and recess of constance to that is to all have more of the seales the mayer, it and peader, but the fore a other say be the sill may eetachation to be state of the somean and so to the something may because we is such all as the
say all to